In [37]:
from sqlalchemy import create_engine
import pandas as pd
from config import password

In [38]:
# create SQL database connection
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'project2'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [7]:
# check for tables
engine.table_names()

['weather', 'bikeshare']

In [8]:
# set path to weather data
path = "toronto_weather_df.csv"

In [10]:
# read in csv file to dataframe
weather = pd.read_csv(path)

In [12]:
weather.head()

,CITY,DATE,PRCP,SNOW_DEPTH,AVG_TEMP,MAX_TEMP,MIN_TEMP
0,"TORONTO CITY, ON CA",01/01/2018,0.01,4.3,8.0,18.0,-1.0
1,"TORONTO CITY, ON CA",01/02/2018,0.00,3.5,14.0,19.0,9.0
2,"TORONTO CITY, ON CA",01/03/2018,0.01,4.3,17.0,22.0,12.0
3,"TORONTO CITY, ON CA",01/04/2018,0.01,4.3,7.0,18.0,-3.0
4,"TORONTO CITY, ON CA",01/05/2018,0.00,3.9,0.0,6.0,-5.0


In [16]:
# create copy of df for transforming
weather_transformed = weather.copy()

In [22]:
# rename columns to match SQL table schema
weather_transformed = weather_transformed.rename(columns={"CITY": "city",
                                                         "DATE": "date_",
                                                         "PRCP": "prcp",
                                                         "SNOW_DEPTH": "snow_depth",
                                                         "AVG_TEMP":"avg_temp",
                                                         "MAX_TEMP":"max_temp",
                                                         "MIN_TEMP":"min_temp"})

In [23]:
weather_transformed.head()

,city,date_,prcp,snow_depth,avg_temp,max_temp,min_temp
0,"TORONTO CITY, ON CA",01/01/2018,0.01,4.3,8.0,18.0,-1.0
1,"TORONTO CITY, ON CA",01/02/2018,0.00,3.5,14.0,19.0,9.0
2,"TORONTO CITY, ON CA",01/03/2018,0.01,4.3,17.0,22.0,12.0
3,"TORONTO CITY, ON CA",01/04/2018,0.01,4.3,7.0,18.0,-3.0
4,"TORONTO CITY, ON CA",01/05/2018,0.00,3.9,0.0,6.0,-5.0


In [27]:
# transform date column to datetime for later merging
weather_transformed['date_'] = pd.to_datetime(weather_transformed['date_'])

In [28]:
weather_transformed.head()

,city,date_,prcp,snow_depth,avg_temp,max_temp,min_temp
0,"TORONTO CITY, ON CA",2018-01-01,0.01,4.3,8.0,18.0,-1.0
1,"TORONTO CITY, ON CA",2018-01-02,0.00,3.5,14.0,19.0,9.0
2,"TORONTO CITY, ON CA",2018-01-03,0.01,4.3,17.0,22.0,12.0
3,"TORONTO CITY, ON CA",2018-01-04,0.01,4.3,7.0,18.0,-3.0
4,"TORONTO CITY, ON CA",2018-01-05,0.00,3.9,0.0,6.0,-5.0


In [30]:
# add weather df to SQL weather table
weather_transformed.to_sql(name = 'weather', con=engine, if_exists='replace', index = False)

In [25]:
# set path and read csv into df
path = "Resources/bikeshare_clean.csv"
bikeshare = pd.read_csv(path)

In [26]:
bikeshare.head()

,trip_id,trip_duration_seconds,date
0,2383648,393,2018-01-01
1,2383649,625,2018-01-01
2,2383650,233,2018-01-01
3,2383651,1138,2018-01-01
4,2383652,703,2018-01-01


In [31]:
# create copy for transforming
bikeshare_transformed = bikeshare.copy()

In [34]:
# rename columns to match SQL table schema
bikeshare_transformed = bikeshare_transformed.rename(columns = {"trip_id":"trip_id",
                                                                "trip_duration_seconds":"trip_duration_seconds",
                                                                "date":"date_"    
})

In [35]:
bikeshare_transformed.head()

,trip_id,trip_duration_seconds,date_
0,2383648,393,2018-01-01
1,2383649,625,2018-01-01
2,2383650,233,2018-01-01
3,2383651,1138,2018-01-01
4,2383652,703,2018-01-01


In [36]:
# add bikeshare df to SQL bikeshare table
bikeshare_transformed.to_sql(name = 'bikeshare', con=engine, if_exists='replace', index = False)